# Libraries & Data Import

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('/kaggle/input/netflix-userbase-dataset/Netflix Userbase.csv', index_col = 'User ID')
df.head(4)

,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
User ID,,,,,,,,,
1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month
2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month
3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month
4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month


# EDA

In [3]:
df.shape

(2500, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 1 to 2500
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Subscription Type  2500 non-null   object
 1   Monthly Revenue    2500 non-null   int64 
 2   Join Date          2500 non-null   object
 3   Last Payment Date  2500 non-null   object
 4   Country            2500 non-null   object
 5   Age                2500 non-null   int64 
 6   Gender             2500 non-null   object
 7   Device             2500 non-null   object
 8   Plan Duration      2500 non-null   object
dtypes: int64(2), object(7)
memory usage: 195.3+ KB


In [5]:
df.nunique()

Subscription Type      3
Monthly Revenue        6
Join Date            300
Last Payment Date     26
Country               10
Age                   26
Gender                 2
Device                 4
Plan Duration          1
dtype: int64

In [6]:
# The avg revenue and age based on the Subscription Type

df.groupby('Subscription Type')[['Monthly Revenue','Age']].aggregate(['mean', 'std','min','max'])

Monthly Revenue                          Age                \
                             mean       std min max       mean       std min   
Subscription Type                                                              
Basic                   12.481481  1.681983  10  15  38.828829  7.190629  27   
Premium                 12.590723  1.733319  10  15  38.512960  7.302325  26   
Standard                12.464844  1.647190  10  15  39.022135  7.020237  27   

                       
                  max  
Subscription Type      
Basic              51  
Premium            51  
Standard           51

In [7]:
# The avg revenue and age from each country based on the Subscription Type

df.groupby(['Subscription Type', 'Country'])[['Monthly Revenue', 'Age']].aggregate(['mean','std','min','max'])

Monthly Revenue                          Age  \
                                            mean       std min max       mean   
Subscription Type Country                                                       
Basic             Australia            12.193548  1.759154  10  15  37.935484   
                  Brazil               12.493151  1.727053  10  15  38.239726   
                  Canada               12.524138  1.671172  10  15  38.393103   
                  France               12.361111  1.457057  10  15  39.277778   
                  Germany              12.302013  1.630390  10  15  38.832215   
                  Italy                12.630682  1.725430  10  15  38.426136   
                  Mexico               11.500000  1.732051  10  13  36.000000   
                  Spain                12.600000  1.649020  10  15  39.718182   
                  United Kingdom       11.666667  2.886751  10  15  48.333333   
                  United States        12.477387  1.690210  10  15  39.412060   
Premium           Australia            12.415842  1.716201  10  15  38.613861   
                  Brazil               12.424242  1.677616  10  15  38.969697   
                  Canada               12.340909  1.639220  10  15  38.488636   
                  France               12.666667  1.713497  10  15  39.068027   
                  Germany              12.000000  2.645751  10  15  43.666667   
                  Italy                13.750000  2.500000  10  15  40.500000   
                  Spain                12.683962  1.746799  10  15  38.367925   
                  United States        12.668966  1.779728  10  15  37.841379   
Standard          Australia            12.529412  1.591891  10  15  38.098039   
                  Brazil               12.750000  1.500000  11  14  36.250000   
                  Canada               12.476190  1.631587  10  15  39.440476   
                  Germany              12.612903  2.060379  10  15  39.580645   
                  Italy                13.000000  0.000000  13  13  43.333333   
                  Mexico               12.240223  1.605241  10  15  38.893855   
                  Spain                12.302326  1.589048  10  15  38.728682   
                  United Kingdom       12.683333  1.676239  10  15  39.038889   
                  United States        12.560748  1.655053  10  15  39.495327   

                                                    
                                       std min max  
Subscription Type Country                           
Basic             Australia       7.680866  27  51  
                  Brazil          7.479909  27  51  
                  Canada          6.876183  27  51  
                  France          6.967726  28  51  
                  Germany         6.652984  27  51  
                  Italy           7.257327  27  51  
                  Mexico          8.869423  27  47  
                  Spain           7.189610  28  51  
                  United Kingdom  2.081666  46  50  
                  United States   7.442782  27  51  
Premium           Australia       7.186056  27  51  
                  Brazil          7.234833  27  51  
                  Canada          7.287202  27  51  
                  France          7.344423  27  51  
                  Germany         6.506407  37  50  
                  Italy           8.103497  30  49  
                  Spain           7.531074  27  51  
                  United States   7.086446  26  51  
Standard          Australia       7.681809  27  51  
                  Brazil          8.180261  29  46  
                  Canada          6.994578  28  51  
                  Germany         6.323365  28  51  
                  Italy           5.507571  38  49  
                  Mexico          6.969832  27  51  
                  Spain           7.557852  27  51  
                  United Kingdom  6.694752  27  51  
                  United States   6.973499  27  51

## Feature engineering

In [8]:
# Date format Y-m-d
# Create a new feature to see if people stoped the Subscription
# If they have not payed since 2023-06-30 then they stoped the subscription (Churned)

df['Join Date'] = pd.to_datetime(df['Join Date'])
df['Last Payment Date'] = pd.to_datetime(df['Last Payment Date'])
df['Churned'] = df['Last Payment Date'].apply(lambda date: 'Yes' if date <= pd.to_datetime('2023-06-30') else 'No')
df

,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration,Churned
User ID,,,,,,,,,,
1,Basic,10,2022-01-15,2023-10-06,United States,28,Male,Smartphone,1 Month,No
2,Premium,15,2021-05-09,2023-06-22,Canada,35,Female,Tablet,1 Month,Yes
3,Standard,12,2023-02-28,2023-06-27,United Kingdom,42,Male,Smart TV,1 Month,Yes
4,Standard,12,2022-10-07,2023-06-26,Australia,51,Female,Laptop,1 Month,Yes
5,Basic,10,2023-01-05,2023-06-28,Germany,33,Male,Smartphone,1 Month,Yes
...,...,...,...,...,...,...,...,...,...,...
2496,Premium,14,2022-07-25,2023-12-07,Spain,28,Female,Smart TV,1 Month,No
2497,Basic,15,2022-04-08,2023-07-14,Spain,33,Female,Smart TV,1 Month,No
2498,Standard,12,2022-09-08,2023-07-15,United States,38,Male,Laptop,1 Month,No


In [9]:
# In this dataset this information is useless
# because the data is so small it doesn't represent the real data
# but if we have a some sizable simple we can find realy interesting things for example
# How many subscribers netflex losses each month?
# What is the revenue each month?
# And alot more, simply we can deal with it as a time series.

df1 = df.copy()
df1['Month'] = df1['Join Date'].dt.to_period('M')
weekly_subscribers = df1.groupby('Month')['Churned'].count()

average_subscribers_per_week = weekly_subscribers.mean()

print("Average number of subscribers per Month:", average_subscribers_per_week)

Average number of subscribers per Month: 83.33333333333333


# Visualization

## What is the Percentage of subscribers from each subscription type?

In [10]:
px.pie(df, names = 'Subscription Type')

## How much does netflex earn each month

In [11]:
histogram = px.histogram(df, x='Monthly Revenue', opacity=0.7)
histogram.update_layout(
    xaxis_title='Monthly Revenue',
    yaxis_title='Count',
    bargap=0.01,
    showlegend=False,
)

histogram.show()

## How many subscribers netfelx has from each conutry?

In [12]:
px.histogram(df, x='Country', barmode = 'group')

## What ages are interested in buying netflex subscription??

In [13]:
histogram = px.histogram(df, x='Age', nbins=10, opacity=0.7)
histogram.update_layout(
    xaxis_title='Age',
    yaxis_title='Count',
    bargap=0.01,
    showlegend=False,
)

histogram.show()

## How about gender who is most likely to buy the subscription

In [14]:
px.histogram(df, x='Gender', barmode = 'group')

## What kind of devices our subscribers uses

In [15]:
px.pie(df, names='Device')

## How about the precentige of people how cancled the subscription

In [16]:
px.pie(df, names='Churned')

## How much do we earn from each subscription type?

In [17]:
px.histogram(df, x='Subscription Type', y='Monthly Revenue', barmode = 'group')

# In each country what is the most popular subscription type?

In [18]:
px.histogram(df, x='Country', color='Subscription Type', barmode = 'group')

## In each age group what is the most popular subscription type?

In [19]:
px.histogram(df, x='Age', color='Subscription Type', barmode = 'group', nbins = 10)

## In each gender what is the most popular subscription type?

In [20]:
px.histogram(df, x='Gender', color='Subscription Type', barmode = 'group')

## In each device what is the most popular subscription type?

In [21]:
px.histogram(df, x='Device', color="Subscription Type", barmode='group')

## What is the most canceld subscription type?

In [22]:
px.histogram(df, x='Churned', color="Subscription Type", barmode='group')

## How much do we earn from each age group?

In [23]:
histogram = px.histogram(df, x='Age', y='Monthly Revenue', barmode = 'group', nbins = 10)
histogram.update_layout(
    bargap=0.01,
    showlegend=False,
)

histogram.show()

## How much do we earn from each counrty?

In [24]:
histogram = px.histogram(df, x='Country', y='Monthly Revenue', barmode = 'group')
histogram.update_layout(
    bargap=0.01,
    showlegend=False,
)
histogram.show()

## How much do we earn from each gender?

In [25]:
histogram = px.histogram(df, x='Gender', y='Monthly Revenue', barmode = 'group')
histogram.update_layout(
    bargap=0.01,
    showlegend=False,
)
histogram.show()

## How much our revenue has decreased?

In [26]:
histogram = px.histogram(df, x='Churned', y='Monthly Revenue', barmode = 'group')
histogram.update_layout(
    bargap=0.01,
    showlegend=False,
)
histogram.show()

## From each country What gender has more subscribers?

In [27]:
px.histogram(df, x='Country', color='Gender', barmode = 'group')

## How much subscribers did we lose from each country?

In [28]:
px.histogram(df, x='Country', color='Churned', barmode = 'group')

## what is the gender with the most unsubscribes?

In [29]:
px.histogram(df, x='Gender', color='Churned', barmode = 'group')